In [33]:
import pandas as pd
import io
from google.cloud import storage
# Create a client
client = storage.Client()

# Define your bucket name
bucket_name = "dsgt-clef-pan-2024"
# Get the bucket
bucket = client.get_bucket(bucket_name)

# List objects in the bucket
blobs = bucket.list_blobs()
for blob in blobs:
    print(blob.name)

JackGraphData/graph_test.tar.gz
JackGraphData/graph_train.tar.gz
JackGraphData/graph_val.tar.gz
PAN Data/Artem.pkl
PAN Data/GPT-wiki-intro.csv
PAN Data/GPT.pkl
PAN Data/HFD.pkl
PAN Data/HF_NabeelShar_ai_and_human_text.csv
PAN Data/HF_artem9k_ai_text_detection_pile/train-00000-of-00007-bc5952582e004d67.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00001-of-00007-71c80017bc45f30d.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00002-of-00007-ee2d43f396e78fbc.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00003-of-00007-529931154b42b51d.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00004-of-00007-b269dc49374a2c0b.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00005-of-00007-3dce5e05ddbad789.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00006-of-00007-3d8a471ba0cf1c8d.parquet
PAN Data/HF_dmitva_human_ai_generated_text.csv
PAN Data/JackGraphData/
PAN Data/Nabeel.pkl
PAN Data/PAN Competition.zip
PAN Data/competition_dataset/datase

In [34]:
from datasets import DatasetDict
file_path = "gs://dsgt-clef-pan-2024/PAN Data/competition_dataset/"
#content = blob.download_as_string()

trainTestValid_dataset = DatasetDict.load_from_disk(file_path)

In [35]:
print(trainTestValid_dataset['train'][0].keys())
print(trainTestValid_dataset['train'][0]['label'])

dict_keys(['text_pair', 'label'])
0


In [36]:
print(trainTestValid_dataset)

DatasetDict({
    train: Dataset({
        features: ['text_pair', 'label'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['text_pair', 'label'],
        num_rows: 15000
    })
    valid: Dataset({
        features: ['text_pair', 'label'],
        num_rows: 15000
    })
})


# Preprocess

In [62]:
from transformers import RobertaTokenizerFast
import pyarrow as pa
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

def tokenize_function(examples):
    tokenized_example = []
    for example, label in zip(examples['text_pair'], examples['label']):
        #print(example)
        tokens = tokenizer(example['text1'], example['text2'], padding='max_length', truncation=True)
        tokens['label']=label
        tokenized_example.append(tokens)
    
    df = pd.DataFrame(tokenized_example)
    #print(tokenized_example)
    tokenized_example = pa.Table.from_pandas(df)
    #print(tokenized_example)
    return tokenized_example
    #return tokenizer(examples['text_pair']['text1'], examples['text_pair']['text2'], padding='max_length', truncation=True)

tokenized_datasets = trainTestValid_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 15000/15000 [01:13<00:00, 204.30 examples/s]


In [63]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 15000
    })
    valid: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 15000
    })
})


In [64]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [65]:
# Get roberta model for classification from Hugging Face
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [67]:
# Print trainable parameters
print(model.print_trainable_parameters())

trainable params: 628,994 || all params: 125,276,164 || trainable%: 0.5020859355176297
None


In [68]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="train/LoRA/", 
    evaluation_strategy="epoch",
    num_train_epochs=20,
    learning_rate=1e-3,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    )

In [69]:
%pip install evaluate
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [70]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [71]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 15000
    })
    valid: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 15000
    })
})


In [74]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = tokenized_datasets['train'].shuffle(seed=42),
    eval_dataset = tokenized_datasets['valid'].shuffle(seed=42),
    compute_metrics=compute_metrics,
)

In [75]:
trainer.train()

KeyboardInterrupt: 

In [76]:
model.save_pretrained("train/LoRA/")

In [77]:
# Run inference on the test dataset
predictions = trainer.predict(tokenized_datasets['test'].shuffle(seed=42))
print(predictions)
#Check the accuracy of predictions
print(compute_metrics(predictions))

PredictionOutput(predictions=array([[-0.05063864,  0.04590682],
       [-0.05064901,  0.04584191],
       [-0.05066186,  0.04588119],
       ...,
       [-0.05066562,  0.04585219],
       [-0.05068301,  0.04590267],
       [-0.05063813,  0.04588834]], dtype=float32), label_ids=array([0, 1, 1, ..., 1, 0, 1]), metrics={'test_loss': 0.6931726932525635, 'test_accuracy': 0.5118, 'test_runtime': 285.2511, 'test_samples_per_second': 52.585, 'test_steps_per_second': 2.191})


ValueError: too many values to unpack (expected 2)

In [78]:
#Print test accuracy
print(predictions)

PredictionOutput(predictions=array([[-0.05063864,  0.04590682],
       [-0.05064901,  0.04584191],
       [-0.05066186,  0.04588119],
       ...,
       [-0.05066562,  0.04585219],
       [-0.05068301,  0.04590267],
       [-0.05063813,  0.04588834]], dtype=float32), label_ids=array([0, 1, 1, ..., 1, 0, 1]), metrics={'test_loss': 0.6931726932525635, 'test_accuracy': 0.5118, 'test_runtime': 285.2511, 'test_samples_per_second': 52.585, 'test_steps_per_second': 2.191})


In [79]:
print(predictions.metrics)

{'test_loss': 0.6931726932525635, 'test_accuracy': 0.5118, 'test_runtime': 285.2511, 'test_samples_per_second': 52.585, 'test_steps_per_second': 2.191}
